In [5]:
"""
 Simple Multi-Layer Pecptron with Custom Data
 @Sungjoon Choi (sungjoon.choi@cpslab.snu.ac.kr)
"""
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline  

In [6]:
# Load them!
cwd = os.getcwd()
loadpath = cwd + "/data/trainingset.npz"
l = np.load(loadpath)

# See what's in here
l.files

# Parse data
trainimg = l['trainimg']
trainlabel = l['trainlabel']
testimg = l['testimg']
testlabel = l['testlabel']
ntrain = trainimg.shape[0]
nclass = trainlabel.shape[1]
dim    = trainimg.shape[1]
ntest  = testimg.shape[0]

print ("%d train images loaded" % (ntrain))
print ("%d test images loaded" % (ntest))
print ("%d dimensional input" % (dim))
print ("%d classes" % (nclass))

69 train images loaded
18 test images loaded
900 dimensional input
2 classes


In [7]:
# Parameters
learning_rate   = 0.001
training_epochs = 100
batch_size      = ntrain
display_step    = 10

# Network Parameters
n_hidden_1 = 10 # 1st layer num features
n_hidden_2 = 10 # 2nd layer num features
n_input    = dim # MNIST data input (img shape: 28*28)
n_classes  = nclass # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# Create model
def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2']))
    return (tf.sigmoid(tf.matmul(layer_2, _weights['out']) + _biases['out']))
    
# Store layers weight & bias
stddev = 0.1 # <== This greatly affects accuracy!! 
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=stddev))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
_pred = multilayer_perceptron(x, weights, biases)

cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(_pred), reduction_indices=1)) # Cross entropy
optm = tf.train.AdamOptimizer(learning_rate).minimize(cost) # Gradient Descent
_corr = tf.equal(tf.argmax(_pred, 1), tf.argmax(y, 1))    
accr = tf.reduce_mean(tf.cast(_corr, tf.float32))
init = tf.initialize_all_variables()

print ("Network Ready")

Network Ready


In [8]:
# Launch the graph
sess = tf.Session()
sess.run(init)

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(ntrain/batch_size)+1
    # Loop over all batches
    for i in range(num_batch): 
        randidx = np.random.randint(ntrain, size=batch_size)
        batch_xs = trainimg[randidx, :]
        batch_ys = trainlabel[randidx, :]                
        # Fit training using batch data
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys})/num_batch

    # Display logs per epoch step
    if epoch % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        train_acc = sess.run(accr, feed_dict={x: batch_xs, y: batch_ys})
        print (" Training accuracy: %.3f" % (train_acc))
        test_acc = sess.run(accr, feed_dict={x: testimg, y: testlabel})
        print (" Test accuracy: %.3f" % (test_acc))

print ("Optimization Finished!")


Epoch: 000/100 cost: 0.852398366
 Training accuracy: 0.348
 Test accuracy: 0.556
Epoch: 010/100 cost: 0.751451105
 Training accuracy: 0.478
 Test accuracy: 0.556
Epoch: 020/100 cost: 0.664153397
 Training accuracy: 0.507
 Test accuracy: 0.556
Epoch: 030/100 cost: 0.581946909
 Training accuracy: 0.391
 Test accuracy: 0.556
Epoch: 040/100 cost: 0.510368854
 Training accuracy: 0.478
 Test accuracy: 0.556
Epoch: 050/100 cost: 0.485279948
 Training accuracy: 0.319
 Test accuracy: 0.556
Epoch: 060/100 cost: 0.410517499
 Training accuracy: 0.493
 Test accuracy: 0.556
Epoch: 070/100 cost: 0.347927839
 Training accuracy: 0.449
 Test accuracy: 0.556
Epoch: 080/100 cost: 0.324010834
 Training accuracy: 0.420
 Test accuracy: 0.556
Epoch: 090/100 cost: 0.266096801
 Training accuracy: 0.507
 Test accuracy: 0.556
Optimization Finished!
